In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.imports import *

In [3]:
PATH = '/home/paperspace/data/materialist/'
sz=224
arch=resnext50
#arch=resnet34
bs=32

In [4]:
label_csv = f'{PATH}labels_2.csv'
label_df = pd.read_csv(label_csv)

In [5]:
n = len(list(open(label_csv))) -1 
val_idxs = get_cv_idxs(n, val_pct=.21)

In [6]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, f'{PATH}merged/train', f'{PATH}labels_2.csv', test_name='test', 
                                    val_idxs=val_idxs, 
                                    tfms=tfms, 
                                    bs=bs)

In [7]:
learn = ConvLearner.pretrained(arch, data, precompute=False, ps=.5)

In [ ]:
learn.save('materialist_50_precomute_true')

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 1e-2
learn.fit(lr, 2, cycle_len=1)

In [ ]:
learn.save('materialist_50_1_precompture')

In [ ]:
learn.precompute = False

In [ ]:
learn.fit(lr, 3, cycle_len=1)

In [ ]:
learn.load('materialist_50_precomp_false')

In [ ]:
learn.unfreeze()

In [ ]:
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 2, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('materialist_50_unfrozen_2')

In [9]:
def get_data(sz):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    return ImageClassifierData.from_csv(PATH, f'{PATH}merged/train', f'{PATH}labels_2.csv', test_name='test', 
                                    val_idxs=val_idxs, 
                                    tfms=tfms, 
                                    bs=bs)

In [10]:
learn.set_data(get_data(500))

In [11]:
learn.freeze()

In [26]:
lr=1e-2
learn.fit(lr, 2, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                        
    0      0.578586   0.551655   0.837592  
    2      0.493885   0.539389   0.840933                          



[0.53938895, 0.8409331330886254]

In [27]:
learn.save('materialist_50_500_3')

In [ ]:
learn.freeze()

In [ ]:
learn.fit(1e-2, 1, cycle_len=1)

In [8]:
learn.load('materialist_50_5000')

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.load('materialist_50_unfrozen_2')

In [ ]:
log_preds,y = learn.TTA(n_aug=4, is_test=True)
preds = np.mean(log_preds, 0)
probs = np.mean(np.exp(log_preds), axis=0)
indices = probs.argmax(axis=1)

In [ ]:
accuracy_np(probs, y)

In [16]:
preds = probs.argmax(axis=1)

In [17]:
indexed_preds = [data.classes[pred] for pred in preds]

In [18]:
def trim_jpeg(filename):
    slash_index = filename.index('/') + 1
    dot_index = filename.index('.')
    return filename[slash_index:dot_index]

In [19]:
def get_filename_indices():
    filenames = data.test_ds.fnames
    filenames = [int(trim_jpeg(filename)) for filename in filenames]
    return filenames

In [20]:
def get_filenames():
    filenames = data.test_ds.fnames
    filenames = [trim_jpeg(filename) for filename in filenames]
    return filenames

In [21]:
def get_missing(indices):
    acc = []
    for x in range(1, 12800):
        if x not in indices:
            acc.append(x)
    return acc

In [22]:
missing_indices = get_missing(get_filename_indices())

In [23]:
filenames = get_filenames()

In [24]:
len(missing_indices)

137

In [25]:
with open(f'{PATH}large_submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(('id', 'predicted'))
    writer.writerows(zip(filenames, indexed_preds))
    for m in missing_indices:
        writer.writerow((m, 1))

In [ ]:
def get_missing(indices):
    acc = []
    for x in range(1, 12800):
        if x not in indices:
            acc.append(x)
    return acc

In [ ]:
def make_submission(preds, incidces):
    missing = get_missing(indices)
    with open(f'{PATH}submission.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(('id', 'predicted'))
        writer.writerows(zip(indices, preds))
        for m in missing:
            writer.writerow((m, 1))

In [ ]:
make_submission(indexed_preds, indices)